In [19]:
import delta_sharing

In [14]:
profile_file = "deltasharing.json"

In [15]:
client = delta_sharing.SharingClient(profile_file)

In [16]:
client.list_all_tables()

[Table(name='table1', share='delta_table', schema='schema1'),
 Table(name='table2', share='delta_table', schema='schema1')]

In [17]:
table_url = profile_file + "#delta_table.schema1.table1"

In [18]:
delta_sharing.load_as_pandas(table_url)

,id
0,6
1,7
2,8
3,5
4,4
5,2
6,3
7,0
8,1
9,9


とりあえず、サーバ起動時にAWSのクレデンシャルを環境変数で与えたが、S3エンドポイントの設定をどうすればよいのか。
サーバのリファレンス実装がHadoopのコンフィグレーションを渡すようになっておらず…。という状況だった。

ひとまず、暫定的にサーバの実装を以下のようにしたら動いた。

io/delta/standalone/internal/DeltaSharedTableLoader.scala:70
```scala
  private val conf = withClassLoader {
    val conf = new Configuration()
    conf.set("fs.s3a.endpoint", "http://127.0.0.1:9000")
    conf.set("fs.s3a.aws.credentials.provider",
      "com.amazonaws.auth.profile.ProfileCredentialsProvider")
    conf
  }
```

あるべき姿という意味では、コンフィグを読み込むようにすべきであるか。